# calculate the specular point on the surface of earth

The idear from Scotte Gleason from the GNSS Applications and Methods, first, the location of receiver and transmitor should be known.

In [2]:
import math
import numpy as np
import scipy.constants as const
from scipy.linalg import norm
import scipy.io as sio    
import matplotlib.pyplot as plt 

In [7]:
# coordinates conversion from WGS-84 to Geodetic coordinates
def xyz2llh(xyz_pos):
    llh=np.zeros(3);
    # constants
    f = 1/298.257223563;        #   WGS-84 Flattening.
    e2 = f*(2.0-f); 
    print(e2)
    a = 6378137.0;              #   WGS-84 equatorial radius (m).                            

    r2=np.dot(xyz_pos[:2],xyz_pos[:2]);
    z=xyz_pos[2];
    zk=0.0;
    while(abs(z-zk)):
        zk=z;
        sinp=z/math.sqrt(r2+z*z);
        v=a/math.sqrt(1.0-e2*sinp*sinp);
        z=xyz_pos[2]+v*e2*sinp;
        
    llh[0]= math.atan(z/math.sqrt(r2)) if r2>1e-12 else (const.pi/2.0 if xyz_pos[2]>0.0 else -const.pi/2.0)
    llh[1]= math.atan2(xyz_pos[1],xyz_pos[0]) if r2>1e-12 else 0.0
    llh[2]= math.sqrt(r2+z*z)-v

    return llh
# calculate the average earth radius at ECEF point
def earth_radius(xyz_pos):
    # WGS-84 constants
    a = 6378137.0;              #   WGS-84 equatorial radius (m).                            
    f = 1/298.257223563;        #   WGS-84 Flattening.
    e2 = f*(2.0-f);             #  e**2
    
    distance=norm(xyz_pos);
    theta=math.asin(xyz_pos[2]/distance);
    temp=1-e2*math.cos(theta)**2;
    
    return a*math.sqrt((1-e2)/temp)
# calculate the average earth radius at ECEF point
def RT2S(r_pos,t_pos):
    # calcuation of earth radius of r_pos
    r_pos=np.array(r_pos);
    t_pos=np.array(t_pos);
    radius=earth_radius(r_pos);
    s=s_temp=r_pos*(radius/norm(r_pos));
    
    k=corr=1e4;
    itera=1;
    while(corr>1e-3):
        unit_vector_S2R=(r_pos-s)/norm(r_pos-s);
        unit_vector_S2T=(t_pos-s)/norm(t_pos-s);
        mid_vector=unit_vector_S2R+unit_vector_S2T;
        s_temp=s+k*mid_vector
        # constain to earth surface
        r=earth_radius(s_temp);
        s_temp=(s_temp/norm(s_temp))*r;
        corr=abs(norm(s_temp-s));
        # new specular point
        s=s_temp;
        # adjust gain
        k=10000 if (corr>10) else 1000;
        itera+=1;
        if itera>1e4:break
    # test Snell's Law
    unit_vector_S2R=(r_pos-s)/norm(r_pos-s);
    unit_vector_S2T=(t_pos-s)/norm(t_pos-s);
    unit_vector_S=s/norm(s);
    incidence=math.acos(np.dot(unit_vector_S2T,unit_vector_S))
    reflectation=math.acos(np.dot(unit_vector_S2R,unit_vector_S))
    diff_angle=abs(incidence-reflectation)*180/const.pi*3600
    return s
def ecef2srcs(Rr,Rt,Rs,v):
    z=Rs/norm(Rs)
    

In [9]:
Rt=-4069896.7033860330,-3583236.9637350840,4527639.2717581640  # postion of transimiter in WGS-84 reference frame
Rr=-11178792.991294,-13160191.204988,20341528.127540         # postion of rreceiver in WGS-84 reference frame

print(RT2S(Rr,Rt))


[-3560138.7832221  -3226742.91309339  4180477.42776175]


TypeError: init() takes 2 positional arguments but 3 were given